### 로직

1. 리뷰 preproecess하고 선별
2. 선별된 리뷰 document로 뭉침
3. 키버트 돌림
4. 감성 분석 돌림
5. 최종 랭킹을 메뉴판과 매칭

### Setting

In [1]:
import pandas as pd

In [2]:
path = "./data/"

rst = pd.read_csv(path + "restaurant.csv", encoding='utf-8-sig')
menu = pd.read_csv(path + "menu.csv", encoding='utf-8-sig')
rev = pd.read_csv(path + "review.csv", encoding='utf-8-sig')

In [3]:
rst.head(3)

,restaurant_id,address,name,opening_hours,phone,type,category,lat,lng,review_count,created_at,updated_at,summary
0,0,서울 광진구 광장로 58-1 1층,범가,[],010-3627-7041,0,중식당,NaN,NaN,NaN,NaN,NaN,NaN
1,1,서울 광진구 워커힐로 177 워커힐 호텔앤리조트,비스타 워커힐 서울 피자힐,[],02-6330-9020,0,피자,NaN,NaN,NaN,NaN,NaN,NaN
2,2,서울 광진구 워커힐로 177 비스타 워커힐 서울 1층,비스타 워커힐 서울 더뷔페,[],02-6330-9015,0,뷔페,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
rst['restaurant_id'].nunique()

3165

In [5]:
menu.head(3)

,menu_id,price,restaurant_id,name,ranking,created_at,updated_at,image_url
0,0,45000,4,새조개 샤브샤브,NaN,NaN,NaN,NaN
1,1,45000,4,하모 샤브샤브,NaN,NaN,NaN,NaN
2,2,10000,4,매생이굴국밥,NaN,NaN,NaN,NaN


In [6]:
menu['restaurant_id'].nunique()

2029

In [7]:
rev.head(3)

,review_id,created_at,user_id,restaurant_id,content,updated_at
0,0,4.2.화,0,0,✨마늘후레이크 찹쌀탕수육 맛보러 가야하는 범가 ✨ 지인이 여기 탕수육 맛있다고 해서...,NaN
1,1,3.11.월,1,0,광나루역맛집 범가에서 마늘찹쌀탕수육 해물짬뽕 해물짜장 먹고 왔어요! 양도 정말 푸짐...,NaN
2,2,2.9.금,2,0,<스티브 추천> 마늘 후레이크 탕수육이 유명하죠. 근데 이번에 알았어요. 여기 ...,NaN


In [25]:
rev['restaurant_id'].nunique()

2656

### 샘플 데이터 뽑기

In [28]:
val_ = rev['restaurant_id'].value_counts()
unique_counts = val_.value_counts()

# 카운트가 40 이상인 60이하만
selected_values = val_[(val_ >= 40) & (val_ <= 60)].index

# 필터링된 값들을 포함하는 원본 데이터프레임 필터링
rev_sample = rev[rev['restaurant_id'].isin(selected_values)]

In [29]:
rev_sample['restaurant_id'].nunique()

137

In [30]:
# rev_sample_df에서 restaurant_id 열의 고유값 추출
unique_restaurant_ids = rev_sample['restaurant_id'].unique()

# menu_df에서 rev_sample_df에 있는 restaurant_id만 남기기
menu_sample = menu[menu['restaurant_id'].isin(unique_restaurant_ids)]

In [36]:
menu_sample['restaurant_id'].nunique()

92

In [34]:
sample = pd.merge(menu_sample, rev_sample, on='restaurant_id', how='inner')

In [42]:
sample.head(2)

,menu_id,price,restaurant_id,name,ranking,created_at_x,updated_at_x,image_url,review_id,created_at_y,user_id,content,updated_at_y
0,492,10000,54,판모밀,NaN,NaN,NaN,NaN,17064,3.15.금,10306,1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요!!!! 아니 회뿐...,NaN
1,492,10000,54,판모밀,NaN,NaN,NaN,NaN,17065,3.19.화,10307,아이들없이 신랑하고 둘만 일식 먹으려고 방문했는데 메뉴 나올때마다 너무 맛있어서 애...,NaN


In [40]:
sample['restaurant_id'].nunique()

92

In [41]:
menu_sample.to_csv(path + "menu_sample.csv", encoding='utf-8-sig')
rev_sample.to_csv(path + "review_sample.csv", encoding='utf-8-sig')

### functions

In [16]:
# 리뷰 테이블 받아서 필요한 리뷰만 선별해 리스트(?) 키버트의 인풋 형식으로 만드는 함수
import re
from kiwipiepy import Kiwi

def drop_short_words(df, threshold):
    # 텍스트가 없는 널값 제거
    df = df.dropna(subset=['content'])

    # 한 단어 이상인 것들만 남기기
    df['word_counts'] = df['content'].apply(lambda x: len(x.split()))

    # 한 단어(threshold) 이상인 것들만 필터링하여 새로운 데이터프레임 반환. 필요없는 word_counts는 삭제
    return df[df['word_counts'] > threshold].drop(columns=['word_counts'])

def preprocess_emoji(content):
    filtered_content = re.sub(r'[^\s\wㄱ-힣\d]', '', content)
    
    return filtered_content

kiwi = Kiwi()
def kiwi_to_sentences(combined_review):

    sentences = kiwi.split_into_sents(combined_review)

    # 출력 형식 중 텍스트만 리스트에 저장
    text_list = [sentence.text for sentence in sentences]
    
    return text_list

def review_table2input(review_table):
    # 필요 컬럼만 선별
    review_table = review_table[['restaurant_id', 'content']]

    # 2단어 이상인 리뷰만 선별 (2단어인 이유는 중간발표 appendix. word count 히스토그램 참고)
    dropped_review_table = drop_short_words(review_table, 1)

    # 'review_text' 열에 정규 표현식 함수 적용
    dropped_review_table['content'] = dropped_review_table['content'].apply(preprocess_emoji)

    # restaurant_id로 그룹화하여 content(review_text)들을 온점 단위로 합치기
    grouped_reviews = dropped_review_table.groupby('restaurant_id')['content'].apply(lambda x: ' . '.join(x)).reset_index()

    # 새로운 데이터프레임 생성 및 문장 단위 분리해 리스트 저장
    new_df = pd.DataFrame(grouped_reviews)
    new_df['review_sentence_split'] = new_df['content'].apply(kiwi_to_sentences)

    return new_df

In [17]:
path = "./data/"

menu_sample = pd.read_csv(path + "menu_sample.csv", encoding='utf-8-sig')
rev_sample = pd.read_csv(path + "review_sample.csv", encoding='utf-8-sig')

preprocessed = review_table2input(rev_sample)

C:\Users\MOONSOO\AppData\Local\Temp\ipykernel_30224\2988013052.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_counts'] = df['content'].apply(lambda x: len(x.split()))


In [20]:
print(len(menu_sample))
print(len(rev_sample))
print(len(preprocessed))

994
6737
137


In [24]:
preprocessed.iloc[132, 2]

['국물 닭발이랑 닭똥집 튀김 먹었는데  음식 궁합이 좋고 있게 잘먹었습니다.',
 '안주가 맛있네요',
 '편안하게 놀기 너무 좋아요 단체석도 많고 여러명이서 회식하기 딱 입니다',
 '굿굿 자주 방문 할게용.',
 '안주가 너무 맛있어서 자주 가는 술집인데 할로윈 시즌이라고 예쁘게 꾸며놓은거 너무 이뻐요 ㅠㅠ.',
 '어제 샤베트가 먹고싶어 방문하였는데 기본안주 마카 로니 과자를 주셔서 먹었는데 상했는지 고무맛에 악 취가 났습니다',
 '바로 뱉긴 했는데 너무 기분이 .',
 '너무맛있게 잘먹고왔어요.',
 '일요일이라 오픈한 마땅한 술집이 없어서 들어갔는데 나름 괜찮았음.',
 '맛있고 친절해요.',
 '안주도 맛있도 직원분들도 착해서 좋은듯.',
 '음식도 괜찮구 청결하고 화장실도 깨끗해요.',
 '집앞에 자주오는 단골술집 오늘도 역시 잘 먹고 갑니다',
 '단체로 가기 좋고 강추합니다.',
 '안주는 하나 하나 다 시킨거 맛있고 화장실 쾌적하고 친절은 기본이여서 술이 너무 기깔나게 들어가서 9시까지야가지고 술을 저거밖에 못마신게 아쉽네요 번창하세요',
 '난 이미 단골 예약s. 사장님 너무 친절해서 좋아요',
 '젊어서 그런가 마인드가 통하는거 같애서 좋네요',
 '맛있는 메뉴가 많아서 삼십분동안 고민하는데 작은 서비스 주셔서 감동 받았어요ㅠㅠ',
 '강냉이 이빠이요ㅋㅋ 옆 테이블에서 짜계치 맛있다고 칭찬을 너무해서 짜계치는 꼭 먹으러 다음주에 갈게요 다음번에는 메뉴 정해서 꼭 올게요',
 '잘먹었습니다 .',
 '2번째 방문  사장님 한결같이 친절하시고 다른 안주 시켜봤는데 항상 맛있어요 술집 분위기가 조금 더 좋게 바껴서 더 좋은거 같애요',
 '술 맛이 나요.',
 '동네에 자주가는 술집입니다 안주도 너무 맛있고 가게가 청결해서 좋아요.',
 '주변 추천 받고 갔는데 종강인데도 불구하고 사장님께서 서비스도 많이 주시고 자리마다 방문하셔서 맛있게 감사히 잘 먹었습니다',
 '이야기 해주셔서 너무 감사했습니다',
 '다음에 또 오겠습니다.',
 '존

In [ ]:
# 메뉴 문장 추출

# 메뉴 데이터에서 메뉴 리스트를 먼저 만들어줍니다
# 추후 이 데이터를 활용해 리뷰에서 메뉴 문장만을 추출합니다

import re
from kiwipiepy import Kiwi
kiwi = Kiwi()

def remove_bracket_contents(menu_names):
    cleaned_list = []

    # 괄호 안의 내용을 모두 지우는 정규 표현식 패턴
    pattern = r'[\[\(].*?[\]\)]'

    for menu_name in menu_names:

        # 패턴을 사용하여 텍스트에서 괄호 안의 내용을 모두 지움
        cleaned_text = re.sub(pattern, '', menu_name)

        if cleaned_text is not None:
            cleaned_list.append(cleaned_text)

    return cleaned_list

def split_menu(menu_names):
    menu_sample_list = []
    menu_dict = []

    for menu_name in menu_names:
        org_menu_dict = {}
        split_result = re.split(r'[+\-\s]+', menu_name)

        # 원본 메뉴명에 대해 딕셔너리 키 생성
        if menu_name not in org_menu_dict:
            org_menu_dict[menu_name] = []  # 초기 리스트 생성


        # 띄어쓰기 단위로 잘라진 메뉴 이름 저장시키기
        menu_name_list = []

        tokened_blank_menu_list = []
        for result in split_result:
            if any(char.isdigit() for char in result):
                # 숫자일 경우 무시
                continue
            else:
                # " ", ""으로 조인시킬 메뉴 네임 리스트(임시적인 리스트)
                menu_name_list.append(result)

                # 얻어진 메뉴명을 토크나이즈 해주고 그것도 메뉴 리스트에 포함
                kiwi_tokens = kiwi.tokenize(result)
                tokened_blank_menu = " ".join([token.form for token in kiwi_tokens])
                tokened_blank_menu_list.append(tokened_blank_menu)

            tokened_blank_menu_join = " ".join(tokened_blank_menu_list)
            # 딕셔너리에 원본 메뉴명의 변형본 추가와 메뉴 샘플리스트에 넣기
            org_menu_dict[menu_name].append(tokened_blank_menu_join)
            menu_sample_list.append(tokened_blank_menu_join)

        menu_dict.append(org_menu_dict)

        
        blank_menu = " ".join(menu_name_list)
        contiuous_menu = "".join(menu_name_list)

        menu_sample_list.append(blank_menu)
        menu_sample_list.append(contiuous_menu)

        # 원본 메뉴 명의 변형을 딕셔너리에 저장
        org_menu_dict[menu_name].append(blank_menu)
        org_menu_dict[menu_name].append(contiuous_menu)
        org_menu_dict[menu_name] = list(set(org_menu_dict[menu_name]))

        # +, -, 공백 단위로 띄워준 메뉴 이름을 리스트에 넣어줌
        # split_result = re.split(r'[+\-\s]+', menu_name)
        # menu_sample_list.extend(split_result)

        # for result in split_result:
        #     # 얻어진 메뉴명을 토크나이즈 해주고 그것도 메뉴 리스트에 포함
        #     kiwi_tokens = kiwi.tokenize(result)
        #     menu_sample_list.extend([token.form for token in kiwi_tokens])

    # 리스트를 set으로 변환하여 중복 제거
    menu_sample_list = set(menu_sample_list)
    # set을 다시 리스트로 변환
    menu_sample_list = list(menu_sample_list)
    
    return menu_sample_list, menu_dict

def make_menu_list(df_menu):
    # 'rst_name'을 기준으로 그룹지어서 'menu_name' 열의 모든 값을 리스트로 모으기
    grouped = df_menu.groupby('rst_name').agg({
        'menu_name': lambda x: list(x)
    })
    
    grouped['preprocessed_menu_name'] = grouped['menu_name'].apply(remove_bracket_contents)

    # 'menu_name' 열의 각 값들을 띄어쓰기 단위로 분할하여 리스트로 만들기
    grouped[['menu_name_split', 'org_menu_dict']] = grouped['preprocessed_menu_name'].apply(split_menu).apply(pd.Series)

    return grouped


### inference

In [ ]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel
from keybert import KeyBERT
import ast

In [ ]:
path = "./data/"

menu_sample = pd.read_csv(path + "menu_sample.csv", encoding='utf-8-sig')
rev_sample = pd.read_csv(path + "review_sample.csv", encoding='utf-8-sig')

In [ ]:
# functions
from kiwipiepy import Kiwi

kiwi = Kiwi()

def adverb_remover(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, len_token in result[0][0]:
        if (
            len_token != 1
            and pos.startswith("J") == False
            and pos.startswith("E") == False
            and pos.startswith("MAJ") == False
        ):
            results.append(token)
    return results


In [ ]:
# KeyBERT 로드. (KoBERT 사용)

model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

In [43]:
rev_sample

,review_id,created_at,user_id,restaurant_id,content,updated_at
17064,17064,3.15.금,10306,54,1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요!!!! 아니 회뿐...,NaN
17065,17065,3.19.화,10307,54,아이들없이 신랑하고 둘만 일식 먹으려고 방문했는데 메뉴 나올때마다 너무 맛있어서 애...,NaN
17066,17066,23.10.1.일,10308,54,숙성 사시미 정말 쫀득하고 입에서 살살녹아요. 사장님이 음식 내주시면서 다설명해주...,NaN
17067,17067,23.10.1.일,10309,54,푸짐한 양에 퀄리티 높은 메뉴들로만 구성되어 있어서 손이 가지 않는 스끼다시 가짓수...,NaN
17068,17068,23.7.29.토,10310,54,일식집 중에 단연코 1등 최애 맛집입니다. 사장님이자 주방장님이신데 처음부터 끝까지...,NaN
...,...,...,...,...,...,...
427869,430843,22.11.13.일,1329,2614,굿,NaN
427870,430844,21.10.29.금,1810,2614,굿,NaN
427871,430845,21.11.11.목,19962,2614,ㅎ,NaN
427872,430846,21.7.10.토,136285,2614,굿,NaN
